# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("city_data_export.csv")
city_df.drop(['Unnamed: 0'], axis=1)
city_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dikson,73.51,80.55,-2.18,92.0,100.0,24.43,RU,1.609809e+09
1,1,rikitea,-23.12,-134.97,76.82,74.0,30.0,17.72,PF,1.609808e+09
2,2,jimo,36.39,120.46,28.99,62.0,0.0,11.18,CN,1.609808e+09
3,3,harper,4.38,-7.72,78.76,83.0,75.0,7.31,LR,1.609809e+09
4,4,kijang,35.24,129.21,32.00,71.0,40.0,6.93,KR,1.609808e+09
...,...,...,...,...,...,...,...,...,...,...
590,590,bud,39.45,-86.18,33.01,92.0,100.0,3.60,US,1.609809e+09
591,591,tyukhtet,56.54,89.30,-25.47,86.0,39.0,3.31,RU,1.609809e+09
592,592,fort frances,48.60,-93.40,26.01,92.0,1.0,4.70,CA,1.609809e+09
593,593,sibolga,1.74,98.78,79.95,71.0,97.0,1.70,ID,1.609809e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_df[['Lat','Lng']]
weights= city_df['Humidity']

fig = gmaps.figure(map_type="TERRAIN",center=(40,15),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations,
                                 weights=weights, 
                                 dissipating=False, 
                                 max_intensity=50,
                                 point_radius=2,
                                 )

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
narrow_df = city_df.loc[(city_df['Max Temp']>=70) & 
                        (city_df['Max Temp']<= 85) & 
                        (city_df['Wind Speed']< 10) &
                        (city_df['Humidity']< 80) &
                        (city_df['Cloudiness']< 20)
                        ]
narrow_df.dropna()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,santa fe,-31.63,-60.70,82.00,41.0,12.0,5.44,AR,1.609809e+09
81,81,salalah,17.02,54.09,71.60,35.0,0.0,4.70,OM,1.609809e+09
101,101,kapaa,22.08,-159.32,78.80,74.0,1.0,9.17,US,1.609809e+09
170,170,busselton,-33.65,115.33,73.00,54.0,0.0,3.00,AU,1.609809e+09
245,245,cabo san lucas,22.89,-109.91,73.40,46.0,5.0,5.82,MX,1.609809e+09
247,247,tuxpan,19.55,-103.40,77.00,65.0,5.0,2.91,MX,1.609809e+09
248,248,carnarvon,-24.87,113.63,78.80,78.0,0.0,6.93,AU,1.609809e+09
263,263,bull savanna,17.89,-77.59,72.10,79.0,0.0,6.96,JM,1.609809e+09
302,302,mier,26.43,-99.15,73.00,35.0,1.0,5.55,MX,1.609809e+09
313,313,kapoeta,4.77,33.59,78.73,41.0,0.0,6.98,SS,1.609809e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
radius = 5000 #meters
location_type = 'hotel'

base_nearby_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?&type={location_type}&radius={radius}&key={g_key}"




In [31]:
hotel_df = pd.DataFrame(columns=['Hotel Name','City','Lat','Lng'])

def create_hotel_df(narrow_df):
    for index,row in narrow_df.iterrows():
        response_json = requests.get(f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={row[2]},{row[3]}&radius={radius}&type={location_type}&&key={g_key}").json()
        #print(json.dumps(response_json, indent=4, sort_keys=True))
    
        yield [response_json['results'][1]['name'],
                response_json['results'][0]['vicinity'],
                response_json['results'][1]['geometry']['location']['lat'],
                response_json['results'][1]['geometry']['location']['lng']
                ]

hotel_df = pd.DataFrame(list(create_hotel_df(narrow_df)),columns=['Hotel Name','City','Lat','Lng'])

hotel_df

,Hotel Name,City,Lat,Lng
0,Conquistador Hotel,Santa Fe,-31.644955,-60.705344
1,HAMDAN PLAZA HOTEL,Salalah,17.019341,54.061011
2,Sheraton Kauai Resort at Coconut Beach,Kapaʻa,22.056756,-159.322517
3,Observatory Guest House,Busselton,-33.646393,115.348676
4,Hotel Tesoro Los Cabos,Cabo San Lucas,22.880938,-109.911797
5,Desarrollo WEBMG,Tuxpan,19.552522,-103.377352
6,Hospitality Carnarvon,Carnarvon,-24.892341,113.655034
7,Big Red's,Alligator Pond,17.882370,-77.597237
8,Rodríguez Barrera Casa Funeral,Ciudad Mier,26.421114,-99.147549
9,Kapoeta Police Station,Kapoeta,4.769777,33.591025


In [13]:
print(response_json['results'][1])
print(response_json['results'][1]['geometry']['location']['lat'])
print(response_json['results'][1]['geometry']['location']['lng'])
print(response_json['results'][1]['name'])
print(response_json['results'][0]['vicinity'])

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 22.8809379, 'lng': -109.9117967}, 'viewport': {'northeast': {'lat': 22.8823916302915, 'lng': -109.9106242697085}, 'southwest': {'lat': 22.8796936697085, 'lng': -109.9133222302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Tesoro Los Cabos', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103521111077615559059">Tesoro Los Cabos</a>'], 'photo_reference': 'ATtYBwJR2BiLS6xku35JwO8ur9c0AjcFqjk5Uzx8N1mTif_CCkc5QPsmGM7cfP_JjHbjgagRxshJPmW-VQHHWWEtlihh5oCZfmYov7NhYNREkrZLxsJJme1KlysBJsxB1nqhSLFAEDsBE_QImTyW9N86nanPmDpy_vNgnZ95fFv1P355ghQd', 'width': 6000}], 'place_id': 'ChIJ37QWDIpcMYQROEF4_EyUYq4', 'plus_code': {'compound_code': 'V3JQ+97 Cabo San Lucas, Baja California Sur, Mexico', 'global_code': '75JGV3JQ+97'}, 'rating': 4.4, 'reference': 'ChIJ37QWDIpcMYQROEF4_EyUYq4', 'scope': 

In [35]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
#<dt>Country</dt><dd>{Country}</dd>
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
